In [ ]:
import beamtest_analysis_helper as helper
from natsort import natsorted
from pathlib import Path
from glob import glob
import hist
import pandas as pd
def return_hist_new(
        input_df: pd.DataFrame,
        chipNames: list[str],
        chipLabels: list[int],
        hist_bins: list = [50, 64, 64]
):
    h = {chipNames[board_idx]: hist.Hist(hist.axis.Regular(hist_bins[0], 100, 240, name="CAL", label="CAL [LSB]"),
                hist.axis.Regular(hist_bins[1], 0, 512,  name="TOT", label="TOT [LSB]"),
                hist.axis.Regular(hist_bins[2], 0, 1024, name="TOA", label="TOA [LSB]"),
        )
    for board_idx in range(len(chipLabels))}

    for board_idx in range(len(chipLabels)):
        tmp_df = input_df.loc[input_df['board'] == chipLabels[board_idx]]
        h[chipNames[board_idx]].fill(tmp_df['cal'].values, tmp_df['tot'].values, tmp_df['toa'].values)

    return h

In [ ]:
# files = glob('/media/daq/X9/Run_nominal_alignment_12/loop_[0-2]/*nem')
# #files = natsorted(files)
# #files = files[30:90]
# for file in files: print(file)
# df = helper.toSingleDataFrame_newEventModel(files=files)

In [ ]:
files = natsorted(list(Path('/media/daq/X9/Run_DESYFeb2024_Run_48/').glob('loop_0/*.bin')))
# files += natsorted(list(Path('/media/daq/X9/Run_DESYFeb2024_Run_20/').glob('loop_1[0-9]/*.bin')))
for file in files: print(file)
decode = helper.DecodeBinary(firmware_key=0b0001, board_id=[0x17f0f, 0x17f0f, 0x17f0f, 0x17f0f], file_list=files)
df = decode.decode_files()[0]
df.info()

In [ ]:
chip_labels = [0, 1, 2, 3]
chip_names = ["ET2_EPIR_Pair1", "ET2p01_BAR_4", "ET2p01_BAR_5", "ET2_EPIR_Pair4"]
# chip_labels = [0, 1, 2]
# chip_names = ["ET2_EPIR_Pair1", "ET2p01_BAR_4", "ET2_EPIR_Pair4"]
# offsets = [20, 20, 20, 20]
offset = 20
offsets = [offset for chiplabel in chip_labels]
high_voltages = [250, 260, 210, 260]
# high_voltages = [250, 250, 250]

fig_path = './'

chip_fignames = chip_names
chip_figtitles = [
    f"(Trigger) Pair1 HV{high_voltages[0]}V OS:{offsets[0]}",
    f"Bar4 HV{high_voltages[1]}V OS:{offsets[1]}",
    f"Bar5 HV{high_voltages[2]}V OS:{offsets[2]}",
    f"Pair4 HV{high_voltages[3]}V OS:{offsets[3]}"]
# chip_figtitles = [
#     f"(Trigger) Pair1 HV{high_voltages[0]}V OS:{offsets[0]}",
#     f"Bar4 HV{high_voltages[1]}V OS:{offsets[1]}",
#     f"Pair4 HV{high_voltages[2]}V OS:{offsets[2]}"]

In [ ]:
helper.plot_occupany_map(df, chip_labels,chip_names, chip_figtitles, fig_tag="inclusive")

In [ ]:
h_inclusive = helper.return_hist(df, chip_names, chip_labels, hist_bins=[100, 256, 512])

helper.plot_1d_TDC_histograms(h_inclusive, chip_names[0], chip_fignames[0], chip_figtitles[0], fig_path, save=False,
                                tag="inclusive", fig_tag=", inclusive", slide_friendly=True)

helper.plot_1d_TDC_histograms(h_inclusive, chip_names[1], chip_fignames[1], chip_figtitles[1], fig_path, save=False,
                                tag="inclusive", fig_tag=", inclusive", slide_friendly=True)

helper.plot_1d_TDC_histograms(h_inclusive, chip_names[2], chip_fignames[2], chip_figtitles[2], fig_path, save=False,
                                tag="inclusive", fig_tag=", inclusive", slide_friendly=True)

helper.plot_1d_TDC_histograms(h_inclusive, chip_names[3], chip_fignames[3], chip_figtitles[3], fig_path, save=False,
                                tag="inclusive", fig_tag=", inclusive", slide_friendly=True)

del h_inclusive

## TDC Filtering on Trigger Board

In [ ]:
# CAL TOA TOT
tdc_cuts = {
    0: [205, 225, 400, 500, 100, 250],
}
tdc_filtered_df = helper.tdc_event_selection(df, tdc_cuts_dict=tdc_cuts, select_by_hit=True)

In [ ]:
custom_tag = ",100<TOT<250 400<TOA<500 205<CAL<225"
# h_inclusive = helper.return_hist(df, chip_names, chip_labels, hist_bins=[100, 256, 512])
helper.plot_occupany_map(tdc_filtered_df, chip_labels, chip_names, chip_figtitles, fig_tag=custom_tag)
h_inclusive = return_hist_new(tdc_filtered_df, chip_names, chip_labels, hist_bins=[100, 256, 512])

helper.plot_1d_TDC_histograms(h_inclusive, chip_names[0], chip_fignames[0], chip_figtitles[0], fig_path, save=False,
                                tag="inclusive", fig_tag=custom_tag, slide_friendly=True)